##Fish species classification. Basic model (test)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Import required libraries

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

# Load the TensorBoard notebook extension
%load_ext tensorboard

###Converting Images to MobileNet Format

In [ ]:
Image_Size=56 #4:3
(img_height, img_width)=(224,224)#(Image_Size*3,Image_Size*4)#4:3

def set_ratio(img):
  desired_ratio = (img_height ) /(img_width)
  initial_ratio = img.shape[0] / img.shape[1]

  if initial_ratio > desired_ratio:
    img=np.pad(img,((0,0),(0, round(img.shape[1] *(initial_ratio/ desired_ratio -1. ))),(0,0)),mode='mean')
  else:
    img=np.pad(img,((0, round(img.shape[0] * (desired_ratio / initial_ratio -1.))) ,(0,0),(0,0)),mode='mean')
  return img


####Setting the catalog - the source of images and the generator that converts images into feature vectors

In [ ]:
# Define our example directories and files

train_data_dir='/content/drive/MyDrive/myFish'
batch_size=120
nb_epochs=15

datagen =  tf.keras.preprocessing.image.ImageDataGenerator( rescale=1./255,
    rotation_range = 40,
    zoom_range = 0.5,
    brightness_range=[0.5,1.5],
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function = set_ratio,
    validation_split=0.1) 


train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode= 'categorical',
    subset='training') # set as training data

validation_generator = datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data


classnum = len(train_generator.class_indices)# number of presented categories

pre_trained_model = tf.keras.applications.MobileNetV2(include_top=True,
                                                           input_shape=(img_height, img_width,3),weights='imagenet')
pre_trained_model.trainable=False

In [ ]:
train_generator.class_indices

In [ ]:
last_layer=pre_trained_model.layers[-2]

In [ ]:
for layer in pre_trained_model.layers:
  layer.trainable = False

print('last layer output shape: ', last_layer.output_shape)


#Neural Network


In [ ]:
from tensorflow.keras.optimizers import RMSprop,Adamax
x = layers.Flatten()(last_layer.output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(512, activation='relu')(x)
# Add a final softmax layer for classification by as many classes as folders of pictures you have
x = layers.Dense(classnum, activation='softmax')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = Adamax(lr=0.001),
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])



### Directory for saving model scales

In [ ]:
weight_saving = '/content/drive/MyDrive/fish/'

In [ ]:
init_lr =1e-4
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
      lambda epoch: init_lr * 10**(epoch / 10))

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.999):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True
callback_stopbyneat = myCallback()

callback_tuneLR = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=5, min_lr=1e-7, min_delta=0.001, factor=0.5)
callback_saveweights = tf.keras.callbacks.ModelCheckpoint(filepath=weight_saving+'model3fish.{epoch:02d}-{val_loss:.2f}.h5',
                                                          monitor='val_loss', save_best_only=True,
                                                          save_weights_only=True
                                                          ),
callback_tensorboard = tf.keras.callbacks.TensorBoard(log_dir='./logs')




In [ ]:
model.load_weights('/content/drive/MyDrive/fish/model3fish.12-0.36.h5',skip_mismatch=True, by_name =True)

In [ ]:
# Run this and see how many epochs it should take before the callback
# fires, and stops training at 99.9% accuracy

history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs=20,
    callbacks=[callback_saveweights, callback_tensorboard, callback_tuneLR]
    )



In [ ]:
%tensorboard --logdir logs

In [ ]:
#Checking the first results with real pictures

from google.colab import files
from tensorflow.keras.preprocessing import image
import numpy as np

uploaded = files.upload()


In [ ]:
import scipy.ndimage as sc

vectors=[]
num=len(uploaded)
fig = plt.figure(figsize=[3,3*num])
i=0
axes=[]
for fn in uploaded.keys():
        # Predicting images
        path = '/content/' + fn
        img = image.load_img(path)
        x = image.img_to_array(img)
        x= set_ratio(x)
        # print(x.shape)
        im = np.stack(( sc.zoom(x[:,:,s],img_height/x.shape[0]) for s in range(x.shape[2])),axis=2)
        im = np.expand_dims(im, axis=0)
        vectors.append(im)
        i+=1
        ax = fig.add_subplot(num,1, i) 
        axes.append(ax)
        ax.imshow(img)
        # print(im.shape)

vectors = np.vstack(vectors)
classes = model.predict(vectors/255., batch_size=batch_size)
fish = train_generator.class_indices

answers = np.array(list(fish.keys()))[classes.argmax(axis=1)]
for i in range(num):
  axes[i].set_title(answers[i])


In [ ]:
fish.keys()


In [ ]:
img_height/x.shape[0]

0.7225806451612903

In [ ]:
classes

array([[0.9534407 , 0.04655926],
       [0.9908296 , 0.00917044],
       [0.9915597 , 0.00844033],
       [0.98656   , 0.01344001],
       [0.76847255, 0.23152742],
       [0.25649363, 0.7435063 ]], dtype=float32)